In [1]:
from apscheduler.schedulers.blocking import BlockingScheduler
from pyquery import PyQuery
import pandas as pd

import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# In[2]:
import time
import datetime
#import win32com.client
import numpy as np
import collections
#import pythoncom

import logging

sched = BlockingScheduler()
logging.basicConfig()


# In[3]:

import sys, logging, traceback
import concurrent.futures as cf
from concurrent.futures import ThreadPoolExecutor
from goose import Goose

In [2]:
import sys, logging, traceback
import concurrent.futures as cf
from concurrent.futures import ThreadPoolExecutor
from goose import Goose


In [3]:
class URLProcessor(object):
	g = Goose()

	def __init__(self, max_workers=50):
		self.max_workers = max_workers

	@staticmethod
	def run_extraction(url):
		extracted = None
		try:
			extracted = URLProcessor.g.extract(url = url)
		except Exception as e:
			sys.stderr.write('\n\t' + str(traceback.print_exc()))
			logging.error(e, exc_info=1)
			extracted = None
		return extracted

	def process_url(self, url):
		return URLProcessor.run_extraction(url)

	def process_urls(self, urls, timeout = 600):
		results = {}
		with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
			futures = {executor.submit(self.process_url, url):url for url in urls}
			for future in cf.as_completed(futures, timeout=timeout):
				if future.result() is not None:
					url = futures[future]
					try:
						results[url] = future.result()
					except Exception as e:
						sys.stderr.write('\n\t' + str(traceback.print_exc()))
						logging.error(e, exc_info=1)
		return results
url_processor = URLProcessor()

In [4]:
def prepare_html(liston,color):
    liston_out=list()
    for headline,url in liston:
        if(len(headline)>0):
            out1_jose = "<tr><td BGCOLOR="+color+">" + headline + "<td align=center>" + url +  "</tr>";
            #out1_jose = "<tr><td BGCOLOR=#F7BE81>" + headline + "<td align=center>" + url +  "</tr>";
            liston_out.append(out1_jose)
    return liston_out

In [19]:
def prepare_wsj():
    pq = PyQuery('http://www.wsj.com/news/heard-on-the-street')

    base_url = 'http://www.wsj.com/'
    
    blog_url='http://blogs.wsj.com/'

    urls = []


    for item in pq('.headline-container .headline a').items():
        href  = item.attr['href']
        if base_url in href:
            urls.append(href)
        else:
            if blog_url not in href:
                urls.append(base_url+href)
            else:
                urls.append(href)

    #url_processor = URLProcessor()

    liston_wsj=list()
    
    for elem in urls:
        article=url_processor.process_url(elem)
        liston_wsj.append((article.title,elem))

    return liston_wsj    

    
    

In [6]:
def prepare_bb():
    pq = PyQuery('http://www.bloomberg.com')

    base_url = 'http://www.bloomberg.com/'

    urls = []


    for item in pq('.hero-v6-story__info-container .hero-v6-story__info a').items():
        href  = item.attr['href']
        if base_url in href:
            urls.append(href)
        else:
            urls.append(base_url+href)

    liston_bb=list()


    for elem in urls:
        article=url_processor.process_url(elem)
        liston_bb.append((article.title,elem))

    return liston_bb

In [7]:
def prepare_cnn():
    pq = PyQuery("http://money.cnn.com/news/economy/")


    #
    base_url = 'http://money.cnn.com/'


    urls = []


    for item in pq('article  a').items():
        href  = item.attr['href']
        if base_url in href:
            urls.append(href)
        else:
            urls.append(base_url+href)

    talla=len(urls)

    limite=min(talla,5)

    liston_cnn=list()



    for elem in urls[0:limite]:
        article=url_processor.process_url(elem)
        liston_cnn.append((article.title,elem))

    return liston_cnn

In [8]:
def prepare_bi():
    pq = PyQuery("http://www.businessinsider.com.au/category/10-things-before-opening-bell")


    #
    base_url = 'http://www.businessinsider.com.au/'


    urls = []


    for item in pq('.post-description .post-title a').items():
        href  = item.attr['href']
        if base_url in href:
            urls.append(href)
        else:
            urls.append(base_url+href)

    talla=len(urls)

    limite=min(talla,2)

    liston_bi=list()



    for elem in urls[0:limite]:
        article=url_processor.process_url(elem)
        liston_bi.append((article.title,elem))

    return liston_bi

In [173]:
def prepare_financiero():
    urls=[]
    texto=[]
    base_url = 'http://www.elfinanciero.com.mx'
    pq = PyQuery('http://www.elfinanciero.com.mx/economia/')

    for item in pq('.category-main-news .title a'):
        #print item.text
        href  = item.attrib['href']
        if base_url in href:
            urls.append(href)
            texto.append(item.text)
        else:
            urls.append(base_url+href)
            texto.append(item.text)
    liston_financiero=list()
    
    for aux  in range(len(urls)):
        liston_financiero.append((texto[aux],urls[aux]))
    return  liston_financiero  
        
        
        
        
        
    


In [170]:
def prepare_universal():
    urls=[]
    urls=[]
    texto=[]
    base_url='http://www.eluniversal.com.mx'
    pq = PyQuery('http://www.eluniversal.com.mx/cartera/economia/')
    
    #for item in pq(' .field-content a'):
    for item in pq('.field-content a'):
        href  = item.attrib['href']
        if href is None:
            continue
        if item.text is None:
            continue
        if len(item.text)<10:
            continue

        if base_url in href:
            urls.append(href)
            texto.append(item.text)
        else:
            urls.append(base_url+href)
            texto.append(item.text)
            
    liston_universal=list()
    
    for aux  in range(len(urls)):
        liston_universal.append((texto[aux],urls[aux]))
    return  liston_universal  
        
            
    

In [9]:
def prepare_big_csv(big_liston):
    path='enter your vipin path'
    today=datetime.datetime.now().date()
    df_out=pd.DataFrame(columns=['text','url'])
    contador=0
    for agency in big_liston:
        for news in agency:
            df_out.loc[contador]=news
            contador=contador+1
            
    df_out.to_csv(path+"\\"+"TOP_NEWS_"+today.strftime('%m%d%Y')+".csv",encoding='utf-8')
    print "news done"
           
            
            
        

In [160]:
def prepare_big_html(big_liston,color):
    total = "<html><body><table border=5 BORDERCOLOR=BLUE><CAPTION><b><u>  NAFTA INSIGHT  </u></b></CAPTION>";
    total = total + "<tr><td><b>" + "TITLE" + "</b><td width='230' align=center><b>"  + "URL" + "</b></tr>";
    tempo=prepare_html(big_liston[0],color[0])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem
    tempo=prepare_html(big_liston[1],color[1])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem
    tempo=prepare_html(big_liston[2],color[2])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem

    tempo=prepare_html(big_liston[3],color[3])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem
            
            
    tempo=prepare_html(big_liston[4],color[4])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem
            
    tempo=prepare_html(big_liston[5],color[5])
    if  len(tempo)>0:
        for elem in tempo:
            total=total+elem



    total = total + "</table></body></html>";

    return total
        

In [46]:
def send_email_new(users,total):
	today=datetime.datetime.now()
	text="REAL TEST SAMARITAN "+today.strftime('%m/%d/%Y %H:%M')
	#s = smtplib.SMTP("smtp.jose.corp")
	s = smtplib.SMTP("smtp.gmail.com",587)
	
	me = 'naftainsight1345@gmail.com'
	pwd='emerson1954'
	s.ehlo()
	s.starttls()
	s.login(me, pwd)
    
	you =users
	msg = MIMEMultipart('alternative')
	msg['Subject'] =text
	msg['From'] = me
	#msg['To'] =you
	msg['Bcc'] =you
	
	#html=total
	part1 = MIMEText(text, 'plain')
	part2 = MIMEText(total, 'html','utf-8')

	# Attach parts into message container.
	# According to RFC 2046, the last part of a multipart message, in this case
	# the HTML message, is best and preferred.
	msg.attach(part1)
	msg.attach(part2)
	s.sendmail(me, you.split(";"), msg.as_string().encode('ascii'))
	s.quit()
    print('email sent.')


In [176]:
#brain
liston_bi=prepare_bi()
liston_wsj=prepare_wsj()
liston_bb=prepare_bb()
liston_cnn=prepare_cnn()
liston_financiero=prepare_financiero()
liston_universal=prepare_universal()

big_liston=[liston_universal,liston_financiero,liston_bi,liston_cnn,liston_bb,liston_wsj]
color=['#809ff7','#f780e3','#ff9900','#00FF00','#00FFFF','#F3F781']

total=prepare_big_html(big_liston,color)



#users="jose.pereza@me.com;vipin.anand.cpp@gmail.com;"
users='jose.pereza@me.com;vipin.anand.cpp@gmail.com;japerez20@gmail.com;juanmanuelhec@gmail.com;salumgreco@hotmail.com'
#users="jperez@joseatm.com;KAzmoon@joseatm.com"
send_email_new(users,total)
#prepare_big_csv(big_liston)

In [22]:
pq = PyQuery("http://money.cnn.com/news/economy/")


#
base_url = 'http://money.cnn.com/'


urls = []


for item in pq('article  a').items():
    href  = item.attr['href']
    if base_url in href:
        urls.append(href)
    else:
        urls.append(base_url+href)

talla=len(urls)

limite=min(talla,5)

liston_cnn=list()



for elem in urls[0:limite]:
    article=url_processor.process_url(elem)
    liston_cnn.append((article.title,elem))

    

In [48]:
pq = PyQuery('http://www.wsj.com/news/heard-on-the-street')

base_url = 'http://www.wsj.com/'

blog_url='http://blogs.wsj.com/'

urls = []


for item in pq('.headline-container .headline a').items():
    href  = item.attr['href']
    if base_url in href:
        urls.append(href)
    else:
        if blog_url not in href:
            urls.append(base_url+href)
        else:
            urls.append(href)

#url_processor = URLProcessor()

liston_wsj=list()

for elem in urls:
    article=url_processor.process_url(elem)
    liston_wsj.append((article.title,elem))

In [56]:
article.meta_keywords

u'alzheimer\u2019s,alzheimer\u2019s disease treatment,biotech,drugmakers,drugs,eli lilly,juno therapeutics,pharmaceuticals,solanezumab,Juno Therapeutics,JUNO,Eli Lilly,LLY,Amgen,AMGN,Regeneron Pharmaceuticals,REGN,regulation,government policy,research,development,corporate,industrial news,new product,service testing,commodity,financial market news,biotechnology,medical equipment,supplies,health care,life sciences,biopharmaceuticals,medical,surgical instruments,apparatus,devices,therapeutic devices,equipment'

In [54]:
article.meta_description

u'Eli Lilly\u2019s Alzheimer\u2019s drug trial failure spotlights the industry\u2019s main problem.'

In [28]:
article.meta_description

"Brothers Mike and Stan Zappa invested $5,000 on a small stockpile of 100 Hatchimals. They've sold 30 on eBay and already recouped their investment on this year's hottest toy."

In [29]:
article.meta_keywords

'Hatchimals, toys, Christmas, Hot toys, tech, innovation, Spin Master, Black Friday, Parija Kavilanz, CNNMoney, CNNMoney.com'

In [35]:
article.cleaned_text

u''

In [38]:
article.raw_doc.body.

<Element body at 0x114e3a470>

In [40]:
today=datetime.datetime.now()
text="TOP NEWS PREMARKET AT "+today.strftime('%m/%d/%Y %H:%M')
#s = smtplib.SMTP("smtp.jose.corp")
s = smtplib.SMTP("smtp.gmail.com",587)

me = 'naftainsight1345@gmail.com'
pwd='emerson1954'
s.ehlo()
s.starttls()
s.login(me, pwd)
users='jose.pereza@me.com;vipin.anand.cpp@gmail.com;japerez20@gmail.com;juanmanuelhec@gmail.com'
you =users
msg = MIMEMultipart('alternative')
msg['Subject'] =text
msg['From'] = me
#msg['To'] =you
msg['Bcc'] =you
text="Real Test  Samaritan"
#html=total
part1 = MIMEText(text, 'plain')
part2 = MIMEText(total, 'html','utf-8')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.
msg.attach(part1)
msg.attach(part2)
s.sendmail(me, you.split(";"), msg.as_string().encode('ascii'))
s.quit()

(221, '2.0.0 closing connection 16sm24030283qtn.33 - gsmtp')

In [ ]:
pq = PyQuery('http://www.wsj.com/news/heard-on-the-street')

base_url = 'http://www.wsj.com/'

blog_url='http://blogs.wsj.com/'

urls = []


for item in pq('.headline-container .headline a').items():
    href  = item.attr['href']
    if base_url in href:
        urls.append(href)
    else:
        if blog_url not in href:
            urls.append(base_url+href)
        else:
            urls.append(href)

#url_processor = URLProcessor()

liston_wsj=list()

for elem in urls:
    article=url_processor.process_url(elem)
    liston_wsj.append((article.title,elem))

In [ ]:
pq = PyQuery('http://www.wsj.com/news/heard-on-the-street')


In [57]:
pq = PyQuery('http://www.elfinanciero.com.mx/economia')


In [64]:
pq('.title')

[<div.title>, <div.title>, <div.title>, <h2.title>]

In [90]:
urls=[]
for item in pq('.article-title .title a'):
    href  = item.attr['href']
    #urls.append(href)
    print href
    

In [95]:
urls=[]
for item in pq('.article-title .title a'):
    #href  = item.attr['href']
    #urls.append(href)
    print item.attrib['href']

In [107]:
urls=[]
texto=[]
base_url = 'http://www.elfinanciero.com.mx/economia'

for item in pq('.category-main-news .title a'):
    print item.text
    href  = item.attrib['href']
    if base_url in href:
        urls.append(href)
        texto.append(item.text)
    else:
        urls.append(base_url+href)
        texto.append(texto)
    #print href
    #urls.append(href)
    

Se mantienen precios de gasolina y diésel en diciembre: Hacienda
Entrada de inversión de cartera extranjera cae a su menor nivel desde 2009
Controversia por propuesta sobre salario


In [103]:
urls

['http://www.elfinanciero.com.mx/economia/economia/se-mantienen-precios-de-gasolina-y-diesel-en-diciembre.html',
 'http://www.elfinanciero.com.mx/economia/economia/cae-entrada-de-inversion-de-cartera-extranjera-a-su-menor-nivel-desde-2009.html',
 'http://www.elfinanciero.com.mx/economia/economia/aumento-al-salario-sin-consenso-provocaria-conflictividad-laboral-stps.html']

In [105]:
article=url_processor.process_url(urls[1])

Traceback (most recent call last):
  File "<ipython-input-3-37df37411729>", line 11, in run_extraction
    extracted = URLProcessor.g.extract(url = url)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/__init__.py", line 56, in extract
    return self.crawl(cc)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/__init__.py", line 66, in crawl
    article = crawler.crawl(crawl_candiate)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/crawler.py", line 123, in crawl
    doc = self.get_document(raw_html)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/crawler.py", line 259, in get_document
    doc = self.parser.fromstring(raw_html)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/parsers.py", line 54, in fromstring
    self.doc = lxml.html.fromstring(html)
  File "/opt/anaconda/lib/python2.7/

In [99]:
liston_fn=list()

for elem in urls:
    article=url_processor.process_url(elem)
    liston_fn.append((article.title,elem))

Traceback (most recent call last):
  File "<ipython-input-3-37df37411729>", line 11, in run_extraction
    extracted = URLProcessor.g.extract(url = url)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/__init__.py", line 56, in extract
    return self.crawl(cc)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/__init__.py", line 66, in crawl
    article = crawler.crawl(crawl_candiate)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/crawler.py", line 123, in crawl
    doc = self.get_document(raw_html)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/crawler.py", line 259, in get_document
    doc = self.parser.fromstring(raw_html)
  File "/opt/anaconda/lib/python2.7/site-packages/goose_extractor-1.0.25-py2.7.egg/goose/parsers.py", line 54, in fromstring
    self.doc = lxml.html.fromstring(html)
  File "/opt/anaconda/lib/python2.7/

AttributeError: 'NoneType' object has no attribute 'title'

In [101]:
limite

5

In [98]:
urls

['http://www.elfinanciero.com.mx/economia/economia/se-mantienen-precios-de-gasolina-y-diesel-en-diciembre.html',
 'http://www.elfinanciero.com.mx/economia/economia/cae-entrada-de-inversion-de-cartera-extranjera-a-su-menor-nivel-desde-2009.html',
 'http://www.elfinanciero.com.mx/economia/economia/aumento-al-salario-sin-consenso-provocaria-conflictividad-laboral-stps.html']

In [59]:
pq.html()

u'\n    <head>\n        <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n                                \n\n            <meta http-equiv="Refresh" content="660"/>\n        <!-- Instant Articles Facebook - @cesarsotoalvarez 18.may.2016 -->\n        <meta property="fb:pages" content="16250043914"/>\n        \n            \n    <meta name="description" content="Lo m\xe1s actual del sector econ\xf3mico de M\xe9xico, Latinoam\xe9rcia, Estados Unidos y Europa."/>\n<meta name="keywords" content="Econom\xeda, sector econ\xf3mico, pol\xedtica monetaria, banco central, noticias econ\xf3micas"/>\n<meta name="refresh" content="1200"/>\n<meta name="robot" content="index,follow"/>\n\n\n                <title>    \n                    \n            El Financiero\n    </title>\n                \n        <link href="/css/screen.css?1" rel="stylesheet" type="text/css" media="screen"/>\n\n\n        <link href="/javascripts/lib/jquery/selectbox/jquery.selectbox.css" rel="stylesheet" 

In [122]:
base_url='http://www.eluniversal.com.mx/cartera/economia'

In [123]:
pq = PyQuery(base_url)

In [124]:
pq.html()

u'\n<head>\n<!--[if lt IE 9]><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<![endif]--><meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<link rel="shortcut icon" href="http://www.eluniversal.com.mx/sites/default/files/favicon.png" type="image/png"/>\n<meta property="fb:app_id" content="163316950407128"/>\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no"/>\n<link sizes="76x76" rel="apple-touch-icon-precomposed" href="/sites/all/themes/universal/images/icos/76.png"/>\n<link sizes="120x120" rel="apple-touch-icon-precomposed" href="/sites/all/themes/universal/images/icos/120.png"/>\n<link sizes="152x152" rel="apple-touch-icon-precomposed" href="/sites/all/themes/universal/images/icos/152.png"/>\n<link rel="apple-touch-icon-precomposed" href="/sites/all/themes/universal/images/icos/76.png"/><meta name="abstract" content="El peri\xf3dico de M\xe9xico l\xedder en noticias y clasific

In [156]:
urls=[]
texto=[]
#for item in pq(' .field-content a'):
for item in pq('.field-content a'):
    href  = item.attrib['href']
    if href is None:
        continue
    if item.text is None:
        continue
    if len(item.text)<10:
        continue
        
    if base_url in href:
        urls.append(href)
        texto.append(item.text)
    else:
        urls.append(base_url+href)
        texto.append(item.text)
    #print href

Licitación de 148 canales atrae a 25 interesados
Economía
None
Ford aumenta oferta en pick ups con modelo Raptor 2017 
Economía
None
Pide Anierm intercambio comercial con otros países 
Economía
Retroceden 4.4% las exportaciones en octubre: Inegi
Economía
None
Sufre economía desaceleración
Economía
None
Banxico advierte de fase de desaceleración
Economía
None
Cierra dólar al menudeo en $20.95
Economía
None
Concluye gira de trabajo de SHCP-Pemex en Reino Unido 
Economía
None
Recomienda Sojo explotar información estadística para entender el "efecto Trump"
Economía
None
Retroceden exportaciones durante octubre
Economía
Economía Universal
None
Walmart seguirá pagando $15 por hora en México
None
Por Trump, empresarios de México y EU alistan estrategia


In [157]:
texto

[u'Licitaci\xf3n de 148 canales atrae a 25 interesados',
 'Ford aumenta oferta en pick ups con modelo Raptor 2017 ',
 u'Pide Anierm intercambio comercial con otros pa\xedses ',
 'Retroceden 4.4% las exportaciones en octubre: Inegi',
 u'Sufre econom\xeda desaceleraci\xf3n',
 u'Banxico advierte de fase de desaceleraci\xf3n',
 u'Cierra d\xf3lar al menudeo en $20.95',
 'Concluye gira de trabajo de SHCP-Pemex en Reino Unido ',
 u'Recomienda Sojo explotar informaci\xf3n estad\xedstica para entender el "efecto Trump"',
 'Retroceden exportaciones durante octubre',
 u'Econom\xeda Universal',
 u'Walmart seguir\xe1 pagando $15 por hora en M\xe9xico',
 u'Por Trump, empresarios de M\xe9xico y EU alistan estrategia']

In [158]:
urls

['http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/26/licitacion-de-148-canales-atrae-25-interesados',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/26/ford-aumenta-oferta-en-pick-ups-con-modelo-raptor-2017',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/26/pide-anierm-intercambio-comercial-con-otros-paises',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/26/retroceden-44-las-exportaciones-en-octubre-inegi',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/26/sufre-economia-desaceleracion',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/25/banxico-advierte-de-fase-de-desaceleracion',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/2016/11/25/cierra-dolar-al-menudeo-en-2095',
 'http://www.eluniversal.com.mx/cartera/economia/articulo/cartera/economia/20

In [1]:
pwd

u'/Users/joseantonioperez/Dropbox/NAFTA'